In [56]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import sys
sys.path.append('../jupyter_common/')
import common as dhira
from dhira_plotly import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. One-Hot Encoding

One hot encoding transforms categorical features to a format that works better with classification and regression algorithms.

This just means that you have a vector the length of the number of classes, and the label element is marked with a 1 while the other labels are set to 0.

**One-hot vector:** Represent every word as an $R^{|V|×1}$ vector with all 0s and one 1 at the index of that word in the sorted english language.

In [2]:
import pandas as pd
s = pd.Series(list('abcde'))
pd.get_dummies(s)

,a,b,c,d,e
0,1,0,0,0,0
1,0,1,0,0,0
2,0,0,1,0,0
3,0,0,0,1,0
4,0,0,0,0,1


In [3]:
df = pd.DataFrame({'A': ['a', 'b', 'c'], 'B' : ['b', 'a', 'c']})
one_hot = pd.get_dummies(df['B'])
df = df.drop('B', axis = 1)
df = df.join(one_hot)
df

,A,a,b,c
0,a,0,1,0
1,b,1,0,0
2,c,0,0,1


In [4]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit([[0, 0, 3], [1, 1, 0], [0, 2, 1], [1, 0, 2]])  


print(enc.n_values_)

print(enc.feature_indices_)

enc.transform([[0, 1, 1]]).toarray()

[2 3 4]
[0 2 5 9]


array([[ 1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.]])

**Can we get the relationship between motel and hotel? No!**

$$(w^{hotel})^Tw^{motel} = (w^{hotel})^Tw^{cat} = 0$$

# 2. Bag of words

A good property to extract from a text corpus is the frequency of words.This representation of the data is know as [Bag of Words](https://en.wikipedia.org/wiki/Bag-of-words_model).

A bag of words is usually represented as a dictionary. The key is the word and the value is the number of times a word appears in the text. For example, the sentence "the fox jumps over the lazy dog" would become:

{'the': 2, 'jumps': 1, 'lazy': 1, 'over': 1, 'fox': 1, 'dog': 1}

#### Limitations
However, bag of words doesn't keep information related to the order of words which is problematic for our understanding. For example, can you derive the meaning from the following bag of words?

{'take': 2, 'I': 2, 'for': 2, 'do': 1, 'them': 1, 'neck': 1, 'head': 1, 'the': 1}   
By losing the ordering of the words, we lose important context that is important to understanding. Keep this in mind as you use this technique for sentiment analysis.

In [5]:
from collections import Counter

def bag_of_words(text):
    # TODO: Implement bag of words
    return Counter(text.split())

test_text = 'the quick brown fox jumps over the lazy dog'

print(bag_of_words(test_text))

Counter({'the': 2, 'lazy': 1, 'fox': 1, 'quick': 1, 'jumps': 1, 'brown': 1, 'dog': 1, 'over': 1})


# 3. Word2Vec
Unsupervised learning methods to learn the conteext of the text.  

**Context of a word**:
- The context of a word is the set of $m$ surrounding words. 
- For instance, the $m = 2$ context of the word "fox" in the sentence "The quick brown fox jumped over the lazy dog" is {"quick", "brown", "jumped", "over"}.
- This model relies on a very important hypothesis in linguistics, distributional similarity, the idea that similar words have similar context



**Papers**
- [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf) by Mikolov et.al   
- [Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/pdf/1310.4546.pdf) by Mikolov et.al 

**TensorFlow Materials**
- https://www.tensorflow.org/tutorials/word2vec   
- https://github.com/tensorflow/models/blob/master/tutorials/embedding/word2vec.py

### Continuous bag of words (CBOW) and 

CBOW aims to predict a center word from the surrounding context in
terms of word vectors.

For each word, we want to learn 2 vectors
- v: (input vector) when the word is in the context
- u: (output vector) when the word is in the center

```
    w(t-2) ---|
    w(t-1) ---|
              |---> target word w(t)
    w(t+1) ---|
    w(t+2) ---|
```

Eg: The cat jumped over the puddle.



###  Skip grams
- Skip-gram does the opposite, and predicts the distribution (probability) of context words from a center word
- In Skip-gram model architecture the training objective is to learn word vector representations
that are good at predicting the nearby words.

```
                                     outputs
                               |--->  w(t-2)
                               |--->  w(t-1)
word w(t) ---> projection ---> |  
                               |--->  w(t+1)
                               |--->  w(t+2)
```

#### 2 training methods: 
- **Negative sampling**
    - Negative sampling defines an objective by sampling negative examples
- **Hierarchical softmax**
    - While hierarchical softmax defines an objective using an efficient tree structure to compute probabilities for all the vocabulary

### word2vec Parameter Learning Explained
- https://arxiv.org/pdf/1411.2738.pdf
- https://arxiv.org/pdf/1402.3722.pdf

### TF Model

![](../../data/images/skip_gram.png)


V – Vocabulary Size (Number of unique words in the corpora)   
P – The Projection or the Embedding Layer   
D – Dimensionality of the Embedding Space  
b – Size of a single Bach   


Eg: `The dog barked at the mailman.` 

**Left Side Model**    
We can visualize the first model as a model that is being trained on data such as 
**(input:'dog',output:['the','barked','at','the','mailman'])** while sharing weights and biases of the softmax layer. In other words, the conceptual model trains multiple outputs to the same input simultaneously. However, this is practically difficult to implement.  

 **Right Side Model**   
A more practical solution would be to break the above tuple 
**(input:'dog', output:['the','barked','at','the','mailman'])** to several single **(input,output)** tuples as **(input:'dog', output:'the'),(input:'dog', output:'barked'),...,(input:'dog', output:'mailman')**. This is what is illustrated in the right-sided model.



#### [NCE Loss](../neuran_network/Loss Functions.ipynb)


In [72]:
import collections
import math
import os
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

### Step 1: Download the data.

In [73]:
url = 'http://mattmahoney.net/dc/'
filename = dhira.maybe_download(url, 'text8.zip', 31344016)

Found and verified ../../data/text8.zip


In [74]:
# Read the data into a list of strings.
def read_data(filename):
  """Extract the first file enclosed in a zip file as a list of words."""
  with zipfile.ZipFile(filename) as f:
    data = tf.compat.as_str(f.read(f.namelist()[0])).split()
  return data

vocabulary = read_data(filename)
print('Data size', len(vocabulary))

Data size 17005207


In [75]:
# vocabulary

### Step 2: Build the dictionary and replace rare words with UNK token.

- Consider only the unique terms/words
- Create a dictionary of words and their index, with UNKNOWN(UNK) as first words with an index -1
    - Eg: {'UNK': 0, 'a': 1, 'the' : 2, ...}
- Convert all words to their coresponding indices with above dictionary    
- Create an inverse dictionary of indices and coresponding words
    - Eg: {0 : 'UNK', 1 : 'a', 2 : 'the', ...}


In [76]:

vocabulary_size = 50000


def build_dataset(words, n_words):
    """
    Process raw inputs into a dataset.
    Some part part of the dataset i.e top 50K is used as the dictionary words 
    and rest as unknown words for simplicity.
    
    """

    # UNK token is used to denote words that are not in the dictionary
    count = [['UNK', -1]]
    #Lets consider only the top most common words for our dictionary and drop the rest 
    # returns set of tuples (word,count) with most common 50000 words
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    # set word count for all the words to the current number of keys in the dictionary
    # in other words values act as indices for each word
    # first word is 'UNK' representing unknown words we encounter
    for word, _ in count:
        dictionary[word] = len(dictionary)
    # this contains the words replaced by assigned indices
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1 #Since this was not accounted in above counter, we do it manually
        data.append(index)
        
    print('Number of unknowns: ', unk_count)  
    count[0][1] = unk_count #Manually counted unknown count is added here
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,
                                                            vocabulary_size)
del vocabulary  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

data_index = 0


Number of unknowns:  418391
Most common words (+UNK) [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)]
Sample data [5235, 3082, 12, 6, 195, 2, 3134, 46, 59, 156] ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']


#### Step 3: Function to generate a training batch for the skip-gram model.


#### Data Generation
Word2Vec is a **unsupersied model**, however neural network is a **supervised model** isn't?
Are we gone mad? No!!!

Lets use the big unstructured data/corpus in our disposal to create the required input and outpus with some trick.

Skip gram depends on two parameters namely:
- **num_skips**: How many times to reuse an input to generate a label.
- **skip_window**: How many words to consider left and right.

```
data: ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first']

with num_skips = 2 and skip_window = 1:
    batch: ['originated', 'originated', 'as', 'as', 'a', 'a', 'term', 'term']
    labels: ['as', 'anarchism', 'originated', 'a', 'term', 'as', 'a', 'of']

with num_skips = 4 and skip_window = 2:
    batch: ['as', 'as', 'as', 'as', 'a', 'a', 'a', 'a']
    labels: ['anarchism', 'term', 'a', 'originated', 'as', 'of', 'originated', 'term']
```    

In [102]:
def generate_batch(batch_size, num_skips, skip_window):
    """
    @params
    batch_size
    num_skips: How many times to reuse an input to generate a label.
    skip_window: How many words to consider left and right.
    """
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window

    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)

    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    # queue which add and pop at the end
    buffer = collections.deque(maxlen=span)
    
    #get words starting from index 0 to span
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data) #To make sure we dont go out of data buffer index 
        
    # num_skips => # of times we select a random word within the span?
    # batch_size (8) and num_skips (2) (4 times)
    # batch_size (8) and num_skips (1) (8 times)
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [skip_window]
        # do this num_skips (2 times)
        # do this (1 time)
        for j in range(num_skips):
            while target in targets_to_avoid:
                # find a target word that is not the word itself
                # while loop will keep repeating until the algorithm find a suitable target word
                target = random.randint(0, span - 1)
            # add selected target to avoid_list for next time
            targets_to_avoid.append(target)
            # e.g. i=0, j=0 => 0; i=0,j=1 => 1; i=1,j=0 => 2
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
        
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

for num_skips, skip_window in [(2, 1), (4, 2), (8,4)]:
    for batch_size in [8, 16]:
        data_index = 0
        batch, labels = generate_batch(batch_size=batch_size, num_skips=num_skips, skip_window=skip_window)
        print('\nwith num_skips = %d, skip_window = %d and batch_size = %d:' % (num_skips, skip_window, batch_size))
        print('    data:', [reverse_dictionary[di] for di in data[:batch_size]])
        print('    batch:', [reverse_dictionary[bi] for bi in batch])
        print('    labels:', [reverse_dictionary[li] for li in labels.reshape(batch_size)])
        print('\n')


with num_skips = 2, skip_window = 1 and batch_size = 8:
    data: ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first']
    batch: ['originated', 'originated', 'as', 'as', 'a', 'a', 'term', 'term']
    labels: ['anarchism', 'as', 'a', 'originated', 'as', 'term', 'a', 'of']



with num_skips = 2, skip_window = 1 and batch_size = 16:
    data: ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the']
    batch: ['originated', 'originated', 'as', 'as', 'a', 'a', 'term', 'term', 'of', 'of', 'abuse', 'abuse', 'first', 'first', 'used', 'used']
    labels: ['anarchism', 'as', 'a', 'originated', 'as', 'term', 'of', 'a', 'term', 'abuse', 'of', 'first', 'used', 'abuse', 'against', 'first']



with num_skips = 4, skip_window = 2 and batch_size = 8:
    data: ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first']
    batch: ['as', 'as', 'as', 'as', 'a', 'a', 'a', 'a']
    

In [83]:
batch

array([12, 12, 12, 12,  6,  6,  6,  6], dtype=int32)

#### Step 4: Build and train a skip-gram model.

$$
\begin{align}
P(w_t | h) &= \text{softmax} (\text{score} (w_t, h)) \\
           &= \frac{\exp \{ \text{score} (w_t, h) \} }
             {\sum_\text{Word w' in Vocab} \exp \{ \text{score} (w', h) \} }
\end{align}
$$
where...
$$
s_θ(w; h) = q'^Tq_w + b_w
$$


Mathematically, the objective (for each example) is to maximize

$$
J_\text{NEG} = \log Q_\theta(D=1 |w_t, h) +
  k \mathop{\mathbb{E}}_{\tilde w \sim P_\text{noise}}
     \left[ \log Q_\theta(D = 0 |\tilde w, h) \right]
$$

In [112]:
np.random.choice?

In [12]:


batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
validation_size = 16     # Random set of words to evaluate similarity on.
validation_window = 100  # Only pick dev samples in the head of the distribution.
validation_examples = np.random.choice(validation_window, validation_size, replace=False)
num_negative_samples = 64    # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default():

    # Input data.
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    validation_dataset = tf.constant(validation_examples, dtype=tf.int32)

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/cpu:0'):
        # Look up embeddings for inputs.
        embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))

        embed = tf.nn.embedding_lookup(embeddings, train_inputs)

        # Construct the variables for the NCE loss
        nce_weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size],
                                                            stddev=1.0 / math.sqrt(embedding_size)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(
          tf.nn.nce_loss(weights=nce_weights,
                         biases=nce_biases,
                         labels=train_labels,
                         inputs=embed,
                         num_sampled=num_negative_samples,
                         num_classes=vocabulary_size))

        # Construct the SGD optimizer using a learning rate of 1.0.
        optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

        # Compute the cosine similarity between minibatch examples and all embeddings.
        norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
        normalized_embeddings = embeddings / norm
        
        validation_embeddings = tf.nn.embedding_lookup(normalized_embeddings, validation_dataset)
        similarity = tf.matmul(validation_embeddings, normalized_embeddings, transpose_b=True)

        # Add variable initializer.
        init = tf.global_variables_initializer()



#### Step 5: Begin training.

### Gradient
- $|W|$ number of word in the vocab
- $y$ and $\hat{y}$ be column vectors of shape $|W|$ x 1
- $u_i$  and $v_j$ be the column vectors of shape $D$ X 1 ($D$ = dimension of embeddings)
- $y$ be the one-hot encoded column vector of shape $|W|$ x 1
- $\hat{y}$ be the softmax prediction column vector of shape $|W|$ x 1
- $\hat{y}_i = P(i|c) = \frac{exp(u_i^Tv_c)}{\sum_{w=1}^Wexp(u_w^Tv_c)}$
- Cross entropy loss: $J = -\sum_{i=1}^Wy_ilog({\hat{y_i}})$
- U = [u_1, u_2, ...,u_k, ...u_W] be a matrix composed of ukuk column vectors.

Now, we can write

$$J = - \sum_{i=1}^W y_i log(\frac{exp(u_i^Tv_c)}{\sum_{w=1}^Wexp(u_w^Tv_c)})$$

Simplifying,

$$J = - \sum_{i=1}^Wy_i[u_i^Tv_c - log(\sum_{w=1}^Wexp(u_w^Tv_c))]$$

Now, we know that yy is one-hot encoded, so all its elements are zero except the one at, say, kthkth index. Which means, there's only one non-zero term in the summation above corresponding to ykyk and all others terms in the summation are zeros. So the cost can also be written as:

$$J = -y_k[u_k^Tv_c - log(\sum_{w=1}^Wexp(u_w^Tv_c))]$$

Note: above $y_k$ is 1.

Solving for \frac{\partial J}{\partial v_c}:

$$\frac{\partial J}{\partial v_c} = -[u_k - \frac{\sum_{w=1}^Wexp(u_w^Tv_c)u_w}{\sum_{x=1}^Wexp(u_x^Tv_c)}]$$

Which can be re-arranged as:

$$\frac{\partial J}{\partial v_c} = \sum_{w=1}^W (\frac{exp(u_w^Tv_c)}{\sum_{x=1}^W exp(u_x^Tv_c)}u_w) - u_k$$

Using definition (5), we can rewrite the above equation as:

$$\frac{\partial J}{\partial v_c} = \sum_{w=1}^W (\hat{y}_w u_w) - u_k$$

Now let's see how this can be written in Matrix notation.Note that:

1. $u_k$ can be written as Matrix vector multiplication: $U.y$
2. And $\sum_{w=1}^W (\hat{y}_w u_w)$ is a linear transformation of vectors uwuw in UU scaled by $y^w$ respectively. This again can be written as $U.\hat{y}$

So the whole thing can be succinctly written as:
$$U[\hat{y} -y]$$

Finally, note that we assumed uiuis to be a column vectors. If we had started with row vectors, we would get $U^T[\hat{y} -y]$.

In [13]:

num_steps = 100001

with tf.Session(graph=graph) as session:
    # We must initialize all variables before we use them.
    init.run()
    print('Initialized')

    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = generate_batch(batch_size, num_skips, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0

        # Note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in xrange(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = 'Nearest to %s:' % valid_word
            for k in xrange(top_k):
                close_word = reverse_dictionary[nearest[k]]
                log_str = '%s %s,' % (log_str, close_word)
            print(log_str)
    final_embeddings = normalized_embeddings.eval()



Initialized
Average loss at step  0 :  285.202941895
Nearest to used: buffalo, reclaimed, icd, schrader, grafton, firmus, group, kenyatta,
Nearest to not: spotless, collides, kripke, painted, district, neurotransmitter, bustling, spassky,
Nearest to i: emphasises, insists, mathilde, labourers, abaye, yoshitaka, macarthur, boats,
Nearest to six: decade, plurality, natal, lokasenna, fremantle, peuple, renegade, blica,
Nearest to first: nevis, eigenvalues, syndicalists, nails, epigrams, lifts, ample, caen,
Nearest to a: perfect, angels, thermonuclear, overcrowded, cornish, choirs, leffler, syndrome,
Nearest to his: miracles, stm, loans, tasmania, landowners, cathay, flawed, fj,
Nearest to seven: riddled, castillo, fosters, holyhead, cluster, confucian, discussions, devotees,
Nearest to the: chilling, outages, anime, credited, mccourt, chadwick, introduce, catastrophism,
Nearest to will: wove, inventors, hercules, lodi, courier, boulevard, discontinuity, carlyle,
Nearest to many: cytokines

#### Step 6: Visualize the embeddings.

In [67]:

def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
    assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'
    data = [
    go.Scatter(
        x=dhira.flatten_numpy_array_to_list(low_dim_embs[:,:1]),
        y=dhira.flatten_numpy_array_to_list(low_dim_embs[:,1:]),
        mode='markers+text',
        text=labels
        )
    ]
    fig = go.Figure(data=data)
    offline.iplot(fig)

try:
    # pylint: disable=g-import-not-at-top
    from sklearn.manifold import TSNE
    import matplotlib.pyplot as plt

    tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
    plot_only = 1000
    low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
    labels = [reverse_dictionary[i] for i in xrange(plot_only)]
    plot_with_labels(low_dim_embs, labels)

except ImportError:
    print('Please install sklearn, matplotlib, and scipy to show embeddings.')